In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *
from src.validation import validate_jumpstart_cube, display_validate_results
from src.coherence import analyze_deck_theme_coherence_enhanced
from src.improve import apply_swap

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

# Optimise

In [ ]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

import importlib
import src.process
importlib.reload(src.process)

from src.process import optimize_deck_coherence, clear_swap_history

# Clear any previous swap history to prevent issues with oscillation
clear_swap_history()

coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
total_coherence = sum(result['overall_coherence'] for result in coherence.values())

improvement = True
iteration = 0
max_iterations = 20  # Prevent infinite loops

while improvement and iteration < max_iterations:
    prev_total_coherence = total_coherence
    cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)
    coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
    total_coherence = sum(result['overall_coherence'] for result in coherence.values())
    improvement = total_coherence > prev_total_coherence
    iteration += 1
    print(f"Iteration {iteration}: Total coherence: {total_coherence:.2f} (improvement: {total_coherence - prev_total_coherence:.2f})")
    
    if not improvement:
        print("No further improvements found - optimization complete!")
    elif iteration >= max_iterations:
        print("Maximum iterations reached - stopping optimization")

In [45]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

import importlib
import src.process
importlib.reload(src.process)

from src.process import optimize_deck_coherence, clear_swap_history

# Clear any previous swap history to prevent issues with oscillation
clear_swap_history()

coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
total_coherence = sum(result['overall_coherence'] for result in coherence.values())

improvement = True
iteration = 0
max_iterations = 20  # Prevent infinite loops

while improvement and iteration < max_iterations:
    prev_total_coherence = total_coherence
    cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)
    coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
    total_coherence = sum(result['overall_coherence'] for result in coherence.values())
    improvement = total_coherence > prev_total_coherence
    iteration += 1
    print(f"Iteration {iteration}: Total coherence: {total_coherence:.2f} (improvement: {total_coherence - prev_total_coherence:.2f})")
    
    if not improvement:
        print("No further improvements found - optimization complete!")
    elif iteration >= max_iterations:
        print("Maximum iterations reached - stopping optimization")

Swap history cleared


Analyzing deck: Green Big Creatures

Current coherence: 5.4

Expected themes: Big Creatures

Deck colors: G

...

- **Llanowar Elves** (Theme Score: 0.0, CMC: 1.0)

### Cards to Add:

- **Bannerhide Krushok** (Theme Score: 4.0) - from Green Stompy

- **Hazard of the Dunes** (Theme Score: 3.0) - from Green Stompy

Coherence change: 5.42 → 6.83 (improvement: 1.41)
✅ Improved Green Big Creatures coherence from 5.4 to 6.8
Iteration 1: Total coherence: 286.63 (improvement: -0.79)
No further improvements found - optimization complete!


# Save to file

In [46]:
from src.export import export_cube_to_csv

export_cube_to_csv(cube_df, oracle_df, 'JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

Exporting cube to JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv...
✅ Successfully exported 390 cards to JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv

📊 Export Summary:
Total cards: 390
Number of decks: 30

Deck breakdown:
  Azorius Evasion/Flying: 13 cards
  Blue Card Draw: 13 cards
  White Equipment: 13 cards
  White Control: 13 cards
  White Aggro: 13 cards
  Selesnya Control: 13 cards
  Red Burn: 13 cards
  Rakdos Burn/Damage: 13 cards
  Orzhov Control: 13 cards
  Red Artifacts: 13 cards
  ... and 20 more decks


'JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv'

# Analysis

In [4]:
# from src.coherence import display_coherence_analysis_enhanced

# display_coherence_analysis_enhanced(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df))

# Generated

In [35]:
# Find the best unassigned cards for Green Stompy
print("🌲 FINDING BEST UNASSIGNED CARDS FOR GREEN STOMPY")
print("=" * 60)

# First, let's reload the updated theme criteria
import importlib
import src.consts
importlib.reload(src.consts)
from src.consts import theme_keywords, theme_criteria

# Get current cards in the cube
cube_card_names = set(cube_df['Name'].tolist())

# Find all cards that are in oracle but not in cube (unassigned)
unassigned_cards = oracle_df[~oracle_df['name'].isin(cube_card_names)]

print(f"Total unassigned cards: {len(unassigned_cards)}")

# Get Green Stompy theme details
stompy_keywords = theme_keywords['Stompy']
stompy_criteria = theme_criteria['Stompy']

print(f"\nGreen Stompy keywords: {stompy_keywords}")
print(f"Power threshold: {stompy_criteria['power_threshold']}")

# Filter for green cards that match Stompy criteria
green_unassigned = unassigned_cards[
    (unassigned_cards['Color Category'].str.contains('Green', na=False))
]

print(f"Green unassigned cards: {len(green_unassigned)}")

# Score each green card for Stompy theme
stompy_candidates = []

for _, card in green_unassigned.iterrows():
    oracle_text = card['Oracle Text'] if pd.notna(card['Oracle Text']) else ''
    card_name = card['name']
    cmc = card['CMC'] if pd.notna(card['CMC']) else 0
    power = card['Power'] if pd.notna(card['Power']) else 0
    
    # Calculate theme score based on keywords
    theme_score = 0
    keyword_matches = []
    
    for keyword in stompy_keywords:
        if keyword.lower() in oracle_text.lower():
            theme_score += 1
            keyword_matches.append(keyword)
    
    # Bonus for meeting power threshold
    if power >= stompy_criteria['power_threshold']:
        theme_score += 2
        keyword_matches.append(f"power>={stompy_criteria['power_threshold']}")
    
    # Bonus for trample (key Stompy ability)
    if 'trample' in oracle_text.lower():
        theme_score += stompy_criteria.get('trample_bonus', 2)
        keyword_matches.append("trample_bonus")
    
    # Bonus for efficient creatures (power >= CMC)
    if power > 0 and power >= cmc:
        theme_score += 1
        keyword_matches.append("efficient")
    
    # Bonus for low CMC aggressive cards
    if cmc <= 3 and power >= 2:
        theme_score += 1
        keyword_matches.append("aggressive")
    
    if theme_score > 0:
        stompy_candidates.append({
            'name': card_name,
            'cmc': cmc,
            'power': power,
            'toughness': card['Toughness'] if pd.notna(card['Toughness']) else 0,
            'type': card['Type'],
            'oracle_text': oracle_text,
            'theme_score': theme_score,
            'matches': keyword_matches
        })

# Sort by theme score
stompy_candidates.sort(key=lambda x: x['theme_score'], reverse=True)

print(f"\n🏆 TOP GREEN STOMPY CANDIDATES:")
print("=" * 50)

for i, candidate in enumerate(stompy_candidates[:15]):  # Show top 15
    print(f"\n{i+1}. {candidate['name']} (Score: {candidate['theme_score']})")
    print(f"   CMC: {candidate['cmc']}, P/T: {candidate['power']}/{candidate['toughness']}")
    print(f"   Type: {candidate['type']}")
    print(f"   Matches: {', '.join(candidate['matches'])}")
    print(f"   Text: {candidate['oracle_text'][:80]}...")

print(f"\n{'='*50}")
print("🎯 BEST STOMPY ADDITIONS:")
print("=" * 50)

# Categorize the best candidates
best_creatures = [c for c in stompy_candidates[:10] if 'Creature' in c['type']]
best_spells = [c for c in stompy_candidates[:10] if 'Creature' not in c['type']]

print("BEST CREATURES:")
for creature in best_creatures[:5]:
    print(f"  + {creature['name']} ({creature['cmc']} CMC, {creature['power']}/{creature['toughness']})")

print("\nBEST SPELLS:")
for spell in best_spells[:3]:
    print(f"  + {spell['name']} ({spell['cmc']} CMC)")

print(f"\n💡 RECOMMENDATIONS:")
print("Consider adding these high-scoring cards to improve Green Stompy coherence!")
print("Focus on creatures with trample and efficient power/CMC ratios.")

🌲 FINDING BEST UNASSIGNED CARDS FOR GREEN STOMPY
Total unassigned cards: 60

Green Stompy keywords: ['trample', 'force', 'green', 'attack', 'overrun', 'aggressive']
Power threshold: 3
Green unassigned cards: 2

🏆 TOP GREEN STOMPY CANDIDATES:

🎯 BEST STOMPY ADDITIONS:
BEST CREATURES:

BEST SPELLS:

💡 RECOMMENDATIONS:
Consider adding these high-scoring cards to improve Green Stompy coherence!
Focus on creatures with trample and efficient power/CMC ratios.


In [36]:
# Let's investigate further - only 2 green unassigned cards seems low
print("🔍 INVESTIGATING UNASSIGNED CARD POOL")
print("=" * 60)

print(f"Oracle cards total: {len(oracle_df)}")
print(f"Cube cards total: {len(cube_df)}")
print(f"Unassigned cards: {len(unassigned_cards)}")

print("\n📊 UNASSIGNED CARDS BY COLOR:")
print("-" * 30)
unassigned_by_color = unassigned_cards['Color Category'].value_counts()
print(unassigned_by_color)

print("\n🌲 THE 2 GREEN UNASSIGNED CARDS:")
print("-" * 40)
for _, card in green_unassigned.iterrows():
    print(f"Card: {card['name']}")
    print(f"Type: {card['Type']}")
    print(f"CMC: {card['CMC']}")
    print(f"P/T: {card['Power']}/{card['Toughness']}")
    print(f"Oracle Text: {card['Oracle Text']}")
    print("-" * 40)

# Let's also check how many green cards are currently in cube
green_cube_cards = cube_df[cube_df['Color Category'].str.contains('Green', na=False)]
print(f"\n🟢 GREEN CARDS CURRENTLY IN CUBE: {len(green_cube_cards)}")

# Let's see the distribution of current green cards by theme
print("\n📈 CURRENT GREEN CARDS BY THEME:")
green_themes = green_cube_cards['Theme'].value_counts()
print(green_themes)

# Check if we should look at all colors for potential additions
print("\n🎯 BEST UNASSIGNED CARDS (ALL COLORS) FOR STOMPY:")
print("=" * 50)

# Score ALL unassigned cards for Stompy theme
all_stompy_candidates = []

for _, card in unassigned_cards.iterrows():
    oracle_text = card['Oracle Text'] if pd.notna(card['Oracle Text']) else ''
    card_name = card['name']
    cmc = card['CMC'] if pd.notna(card['CMC']) else 0
    power = card['Power'] if pd.notna(card['Power']) else 0
    color = card['Color Category'] if pd.notna(card['Color Category']) else ''
    
    # Calculate theme score
    theme_score = 0
    keyword_matches = []
    
    for keyword in stompy_keywords:
        if keyword.lower() in oracle_text.lower():
            theme_score += 1
            keyword_matches.append(keyword)
    
    # Power threshold bonus
    if power >= stompy_criteria['power_threshold']:
        theme_score += 2
        keyword_matches.append(f"power>={stompy_criteria['power_threshold']}")
    
    # Trample bonus
    if 'trample' in oracle_text.lower():
        theme_score += 2
        keyword_matches.append("trample")
    
    # Efficiency bonus
    if power > 0 and power >= cmc:
        theme_score += 1
        keyword_matches.append("efficient")
    
    # Aggressive bonus
    if cmc <= 3 and power >= 2:
        theme_score += 1
        keyword_matches.append("aggressive")
    
    if theme_score > 0:
        all_stompy_candidates.append({
            'name': card_name,
            'color': color,
            'cmc': cmc,
            'power': power,
            'toughness': card['Toughness'] if pd.notna(card['Toughness']) else 0,
            'type': card['Type'],
            'oracle_text': oracle_text,
            'theme_score': theme_score,
            'matches': keyword_matches
        })

all_stompy_candidates.sort(key=lambda x: x['theme_score'], reverse=True)

print(f"Found {len(all_stompy_candidates)} cards with Stompy potential")

for i, candidate in enumerate(all_stompy_candidates[:10]):
    print(f"\n{i+1}. {candidate['name']} (Score: {candidate['theme_score']})")
    print(f"   Color: {candidate['color']}")
    print(f"   CMC: {candidate['cmc']}, P/T: {candidate['power']}/{candidate['toughness']}")
    print(f"   Matches: {', '.join(candidate['matches'])}")
    print(f"   Text: {candidate['oracle_text'][:100]}...")

🔍 INVESTIGATING UNASSIGNED CARD POOL
Oracle cards total: 450
Cube cards total: 390
Unassigned cards: 60

📊 UNASSIGNED CARDS BY COLOR:
------------------------------
Color Category
Lands        26
Colorless    11
White         8
Black         7
Blue          4
Red           2
Green         2
Name: count, dtype: int64

🌲 THE 2 GREEN UNASSIGNED CARDS:
----------------------------------------
Card: Giant Growth
Type: Instant
CMC: 1
P/T: nan/nan
Oracle Text: Target creature gets +3/+3 until end of turn.
----------------------------------------
Card: Hickory Woodlot
Type: Land
CMC: 0
P/T: nan/nan
Oracle Text: This land enters tapped with two depletion counters on it. | {T}, Remove a depletion counter from this land: Add {G}{G}. If there are no depletion counters on this land, sacrifice it.
----------------------------------------


KeyError: 'Color Category'

In [37]:
# Check column names to understand the structure
print("📋 ORACLE_DF COLUMNS:")
print(oracle_df.columns.tolist())
print("\n📋 CUBE_DF COLUMNS:")
print(cube_df.columns.tolist())

# Let's look at a sample from each to understand the structure
print("\n🔍 ORACLE_DF SAMPLE:")
print(oracle_df.head(2)[['name', 'Color Category', 'Type', 'Power', 'Toughness']])

print("\n🔍 CUBE_DF SAMPLE:")
print(cube_df.head(2))

# Now let's properly analyze green cards - check what column to use for color in cube_df
# Look for color-related columns
color_columns = [col for col in cube_df.columns if 'color' in col.lower() or 'Color' in col]
print(f"\n🎨 COLOR-RELATED COLUMNS IN CUBE_DF: {color_columns}")

# Let's see if we can identify green cards in the cube another way
print("\n🌲 ANALYZING GREEN CARDS IN CUBE:")
# Try to find green cards by looking at different potential color columns
if 'Colors' in cube_df.columns:
    green_cube_cards = cube_df[cube_df['Colors'].str.contains('G', na=False)]
    print(f"Green cards by 'Colors' column: {len(green_cube_cards)}")
elif 'Color' in cube_df.columns:
    green_cube_cards = cube_df[cube_df['Color'].str.contains('G', na=False)]
    print(f"Green cards by 'Color' column: {len(green_cube_cards)}")
else:
    # Look at theme distribution instead
    print("Theme distribution in cube:")
    print(cube_df['Theme'].value_counts())
    
    # Filter for Stompy and Big Creatures themes (green themes)
    green_themes = ['Stompy', 'Big Creatures']
    green_cube_cards = cube_df[cube_df['Theme'].isin(green_themes)]
    print(f"Green-themed cards: {len(green_cube_cards)}")

print(f"\n🟢 GREEN CARDS CURRENTLY IN CUBE: {len(green_cube_cards)}")

# Show current green cards
if len(green_cube_cards) > 0:
    print("\n📈 CURRENT GREEN CARDS BY THEME:")
    green_themes_count = green_cube_cards['Theme'].value_counts()
    print(green_themes_count)
    
    print(f"\n🃏 SAMPLE GREEN CARDS:")
    print(green_cube_cards[['Name', 'Theme']].head(10))

📋 ORACLE_DF COLUMNS:
['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID', 'Power', 'Toughness']

📋 CUBE_DF COLUMNS:
['Name', 'Set', 'Collector Number', 'Rarity', 'Color Identity', 'Type', 'Mana Cost', 'CMC', 'Power', 'Toughness', 'Tags']

🔍 ORACLE_DF SAMPLE:
              name Color Category                      Type  Power  Toughness
0      Boros Elite          White  Creature - Human Soldier    1.0        1.0
1  Deftblade Elite          White  Creature - Human Soldier    1.0        1.0

🔍 CUBE_DF SAMPLE:
                    Name    Set Collector Number  Rarity Color Identity  \
0          Archaeomancer  Mixed              NaN  common              U   
1  Behold the Multiverse  Mixed              NaN  common              U   

                      Type Mana Cost  CMC Power Toughness  \
0  Creature - Human Wizard       NaN    4   NaN       NaN   
1                  Instant       NaN    4   NaN       NaN   

                     Tags  
0  Azorius Evasion

KeyError: 'Theme'

In [38]:
# Now I understand the structure better
print("📊 CUBE STRUCTURE ANALYSIS")
print("=" * 50)

# The cube uses 'Tags' for themes and 'Color Identity' for colors
print("🏷️ THEME DISTRIBUTION IN CUBE (Tags column):")
theme_distribution = cube_df['Tags'].value_counts()
print(theme_distribution)

# Find green cards using Color Identity
green_cube_cards = cube_df[cube_df['Color Identity'].str.contains('G', na=False)]
print(f"\n🟢 GREEN CARDS IN CUBE: {len(green_cube_cards)}")

# Show green cards by theme (Tags)
print(f"\n🌲 GREEN CARDS BY THEME:")
green_theme_dist = green_cube_cards['Tags'].value_counts()
print(green_theme_dist)

# Now let's look at the 2 unassigned green cards more carefully
print(f"\n🔍 THE 2 UNASSIGNED GREEN CARDS:")
print("-" * 40)
for _, card in green_unassigned.iterrows():
    print(f"Name: {card['name']}")
    print(f"Type: {card['Type']}")
    print(f"CMC: {card['CMC']}")
    if pd.notna(card['Power']):
        print(f"Power/Toughness: {card['Power']}/{card['Toughness']}")
    print(f"Oracle Text: {card['Oracle Text']}")
    
    # Score this card for Stompy
    oracle_text = card['Oracle Text'] if pd.notna(card['Oracle Text']) else ''
    power = card['Power'] if pd.notna(card['Power']) else 0
    cmc = card['CMC'] if pd.notna(card['CMC']) else 0
    
    stompy_score = 0
    matches = []
    
    for keyword in stompy_keywords:
        if keyword.lower() in oracle_text.lower():
            stompy_score += 1
            matches.append(keyword)
    
    if power >= stompy_criteria['power_threshold']:
        stompy_score += 2
        matches.append(f"power>={stompy_criteria['power_threshold']}")
    
    if 'trample' in oracle_text.lower():
        stompy_score += 2
        matches.append("trample")
    
    print(f"Stompy Score: {stompy_score} (matches: {matches})")
    print("-" * 40)

# Let's also check what Green Stompy cards are currently in the cube
stompy_cards = cube_df[cube_df['Tags'].str.contains('Stompy', na=False)]
print(f"\n🦏 CURRENT STOMPY CARDS IN CUBE: {len(stompy_cards)}")
if len(stompy_cards) > 0:
    print(stompy_cards[['Name', 'Color Identity', 'Tags']].head(10))

# Check for "Big Creatures" theme too
big_creatures = cube_df[cube_df['Tags'].str.contains('Big Creatures', na=False)]
print(f"\n🐘 CURRENT BIG CREATURES CARDS IN CUBE: {len(big_creatures)}")
if len(big_creatures) > 0:
    print(big_creatures[['Name', 'Color Identity', 'Tags']].head(10))

# Summary
print(f"\n📊 SUMMARY:")
print(f"• Total oracle cards: {len(oracle_df)}")
print(f"• Total cube cards: {len(cube_df)}")
print(f"• Unassigned cards: {len(unassigned_cards)}")
print(f"• Green unassigned: {len(green_unassigned)}")
print(f"• Current green cube cards: {len(green_cube_cards)}")
print(f"• Current Stompy cards: {len(stompy_cards)}")
print(f"• Current Big Creatures cards: {len(big_creatures)}")

print(f"\n💡 INSIGHT:")
print("With only 2 unassigned green cards (Giant Growth and Hickory Woodlot),")
print("the oracle database appears to be almost fully utilized in the cube.")
print("Giant Growth could potentially fit Stompy theme as a pump spell.")
print("Consider looking at existing Stompy cards for optimization instead.")

📊 CUBE STRUCTURE ANALYSIS
🏷️ THEME DISTRIBUTION IN CUBE (Tags column):
Tags
Azorius Evasion/Flying    13
Blue Card Draw            13
White Equipment           13
White Control             13
White Aggro               13
Selesnya Control          13
Red Burn                  13
Rakdos Burn/Damage        13
Orzhov Control            13
Red Artifacts             13
Red Aggro                 13
Izzet Control             13
Red Small Creatures       13
Gruul Aggro/Beatdown      13
Green Ramp                13
Green Big Creatures       13
Green Stompy              13
Green Midrange            13
Golgari Graveyard         13
Boros Aggro/Beatdown      13
Simic Control             13
Blue Control              13
Dimir Control             13
Blue Flying               13
Black Sacrifice           13
Black Graveyard           13
Black Control             13
Black Aggro               13
Blue Tempo                13
White Tokens              13
Name: count, dtype: int64

🟢 GREEN CARDS IN CUBE: 89



In [39]:
# Final Green Stompy Recommendations
print("🏆 GREEN STOMPY OPTIMIZATION RECOMMENDATIONS")
print("=" * 60)

print("📋 CURRENT STATUS:")
print(f"• Green Stompy has 12/13 cards (missing 1 card)")
print(f"• Green Big Creatures has 13/13 cards (complete)")
print(f"• Only 2 unassigned green cards available:")
print(f"  - Giant Growth (pump spell)")
print(f"  - Hickory Woodlot (land)")

print(f"\n🎯 RECOMMENDATIONS:")
print("=" * 40)

print("1. 🟢 ADD GIANT GROWTH TO STOMPY:")
print("   • Giant Growth fits Stompy theme as an aggressive pump spell")
print("   • +3/+3 boost can enable aggressive attacks")
print("   • Low CMC (1) fits aggro strategy") 
print("   • Would complete the 13-card Stompy theme")

print("\n2. 🔄 CONSIDER INTERNAL OPTIMIZATION:")
print("   • Current Stompy deck might have suboptimal cards")
print("   • Could swap cards between Stompy and Big Creatures")
print("   • Some cards in Big Creatures might better fit Stompy")

# Let's look at the actual current Stompy cards more closely
print(f"\n📝 DETAILED STOMPY ANALYSIS:")
print("-" * 40)

stompy_cards_full = cube_df[cube_df['Tags'] == 'Green Stompy']
print(f"Current Green Stompy cards ({len(stompy_cards_full)}/13):")

for _, card in stompy_cards_full.iterrows():
    name = card['Name']
    cmc = card['CMC'] if pd.notna(card['CMC']) else 'N/A'
    power = card['Power'] if pd.notna(card['Power']) else 'N/A'
    toughness = card['Toughness'] if pd.notna(card['Toughness']) else 'N/A'
    card_type = card['Type'] if pd.notna(card['Type']) else 'N/A'
    
    print(f"  • {name} ({cmc} CMC, {power}/{toughness}) - {card_type}")

print(f"\n📝 BIG CREATURES FOR COMPARISON:")
print("-" * 40)

big_creatures_full = cube_df[cube_df['Tags'] == 'Green Big Creatures']
print(f"Current Green Big Creatures cards ({len(big_creatures_full)}/13):")

for _, card in big_creatures_full.iterrows():
    name = card['Name']
    cmc = card['CMC'] if pd.notna(card['CMC']) else 'N/A'
    power = card['Power'] if pd.notna(card['Power']) else 'N/A'
    toughness = card['Toughness'] if pd.notna(card['Toughness']) else 'N/A'
    card_type = card['Type'] if pd.notna(card['Type']) else 'N/A'
    
    print(f"  • {name} ({cmc} CMC, {power}/{toughness}) - {card_type}")

print(f"\n💡 FINAL RECOMMENDATION:")
print("=" * 40)
print("🎯 ADD Giant Growth to complete Green Stompy theme!")
print("🔧 Then run optimize_deck_coherence to refine both Green themes.")
print("📊 The optimization algorithm can swap cards between themes as needed.")

# Show how to add Giant Growth
print(f"\n⚡ QUICK ACTION:")
print("To add Giant Growth to Green Stompy:")
print("1. Add 'Giant Growth' to the Green Stompy deck in your cube")
print("2. Run the optimization algorithm") 
print("3. Check if any cards should swap between Stompy and Big Creatures")

🏆 GREEN STOMPY OPTIMIZATION RECOMMENDATIONS
📋 CURRENT STATUS:
• Green Stompy has 12/13 cards (missing 1 card)
• Green Big Creatures has 13/13 cards (complete)
• Only 2 unassigned green cards available:
  - Giant Growth (pump spell)
  - Hickory Woodlot (land)

🎯 RECOMMENDATIONS:
1. 🟢 ADD GIANT GROWTH TO STOMPY:
   • Giant Growth fits Stompy theme as an aggressive pump spell
   • +3/+3 boost can enable aggressive attacks
   • Low CMC (1) fits aggro strategy
   • Would complete the 13-card Stompy theme

2. 🔄 CONSIDER INTERNAL OPTIMIZATION:
   • Current Stompy deck might have suboptimal cards
   • Could swap cards between Stompy and Big Creatures
   • Some cards in Big Creatures might better fit Stompy

📝 DETAILED STOMPY ANALYSIS:
----------------------------------------
Current Green Stompy cards (13/13):
  • Hooting Mandrills (6 CMC, N/A/N/A) - Creature - Ape
  • Massive Might (1 CMC, N/A/N/A) - Instant
  • Rancor (1 CMC, N/A/N/A) - Enchantment - Aura
  • Trumpeting Herd (4 CMC, N/A/N/A)

In [40]:
# Analyze Giant Growth and similar cards to identify missing Stompy keywords
print("🔍 ANALYZING PUMP SPELLS AND STOMPY KEYWORDS")
print("=" * 60)

# Current Stompy keywords
current_stompy_keywords = theme_keywords['Stompy']
print(f"Current Stompy keywords: {current_stompy_keywords}")

# Let's analyze Giant Growth specifically
giant_growth_text = "Target creature gets +3/+3 until end of turn."
print(f"\n📝 Giant Growth text: '{giant_growth_text}'")

# Look for pump spell patterns in the oracle
pump_spells = oracle_df[
    oracle_df['Oracle Text'].str.contains(r'\+\d+/\+\d+', na=False, regex=True)
]

print(f"\n🔧 PUMP SPELLS IN ORACLE ({len(pump_spells)} cards):")
print("-" * 40)

for _, spell in pump_spells.head(10).iterrows():
    print(f"• {spell['name']}: {spell['Oracle Text']}")

# Look for other aggressive/combat keywords that might fit Stompy
combat_keywords = [
    'gets +', 'target creature gets', 'pump', 'boost', 'enhance', 
    'combat', 'attack', 'attacking', 'power', 'toughness',
    'until end of turn', 'this turn', 'trample', 'haste',
    'can\'t be blocked', 'unblockable', 'must be blocked',
    'double strike', 'first strike', 'vigilance'
]

print(f"\n🎯 POTENTIAL STOMPY KEYWORDS TO ADD:")
print("-" * 40)

# Check current Stompy cards for common text patterns
current_stompy_cards = cube_df[cube_df['Tags'] == 'Green Stompy']

print("Analyzing current Stompy cards for keyword patterns...")

# Let's look at the oracle data for current stompy cards to see their text
stompy_oracle_matches = []
for _, stompy_card in current_stompy_cards.iterrows():
    oracle_match = oracle_df[oracle_df['name'] == stompy_card['Name']]
    if not oracle_match.empty:
        stompy_oracle_matches.append({
            'name': stompy_card['Name'],
            'oracle_text': oracle_match.iloc[0]['Oracle Text'] if pd.notna(oracle_match.iloc[0]['Oracle Text']) else ''
        })

print(f"\n📋 CURRENT STOMPY CARDS WITH ORACLE TEXT:")
for match in stompy_oracle_matches[:8]:  # Show first 8
    print(f"• {match['name']}: {match['oracle_text']}")

# Analyze common words/phrases in Stompy cards
all_stompy_text = ' '.join([match['oracle_text'].lower() for match in stompy_oracle_matches])
stompy_words = all_stompy_text.split()

# Count frequency of potentially relevant words
from collections import Counter
word_counts = Counter(stompy_words)

relevant_words = []
for word in combat_keywords:
    count = all_stompy_text.count(word.lower())
    if count > 0:
        relevant_words.append((word, count))

print(f"\n🔤 KEYWORDS FOUND IN CURRENT STOMPY CARDS:")
for word, count in sorted(relevant_words, key=lambda x: x[1], reverse=True):
    print(f"  '{word}' appears {count} times")

# Specific suggestions for Stompy keywords
print(f"\n💡 RECOMMENDED ADDITIONS TO STOMPY KEYWORDS:")
print("-" * 50)

suggested_keywords = [
    'gets +',           # For pump spells like Giant Growth
    'target creature',  # Common in pump/combat spells  
    '+3/+3',           # Specific pump amounts
    '+2/+2', 
    '+1/+1',
    'until end of turn', # Temporary boosts
    'combat',          # Combat-related abilities
    'power',           # Power-matters effects
    'toughness',       # Size-matters effects
    'haste',           # Aggressive keyword
    'vigilance',       # Combat keyword
    'first strike',    # Combat keyword
    'double strike',   # Combat keyword
    'can\'t be blocked', # Evasion for aggro
    'must be blocked'   # Forces combat
]

print("Keywords that would help Giant Growth and similar cards score:")
for keyword in suggested_keywords:
    if keyword not in current_stompy_keywords:
        # Check if this keyword appears in Giant Growth or current Stompy cards
        in_giant_growth = keyword.lower() in giant_growth_text.lower()
        in_stompy = keyword.lower() in all_stompy_text.lower()
        
        marker = "🎯" if in_giant_growth else ("✓" if in_stompy else "○")
        print(f"  {marker} '{keyword}' - {'In Giant Growth' if in_giant_growth else ('In current Stompy' if in_stompy else 'Potential addition')}")

print(f"\n🎯 = Would help Giant Growth score")
print(f"✓ = Already used in current Stompy cards") 
print(f"○ = Potential future addition")

🔍 ANALYZING PUMP SPELLS AND STOMPY KEYWORDS
Current Stompy keywords: ['trample', 'force', 'green', 'attack', 'overrun', 'aggressive']

📝 Giant Growth text: 'Target creature gets +3/+3 until end of turn.'

🔧 PUMP SPELLS IN ORACLE (95 cards):
----------------------------------------
• Boros Elite: Battalion — Whenever this creature and at least two other creatures attack, this creature gets +2/+2 until end of turn.
• Faerie Guidemother: Flying // Target creature gets +2/+1 and gains flying until end of turn. (Then exile this card. You may cast the creature later from exile.)
• Miner's Guidewing: Flying, vigilance | When this creature dies, target creature you control explores. (Reveal the top card of your library. Put that card into your hand if it's a land. Otherwise, put a +1/+1 counter on that creature, then put the card back or put it into your graveyard.)
• Ainok Bond-Kin: Outlast {1}{W} ({1}{W}, {T}: Put a +1/+1 counter on this creature. Outlast only as a sorcery.) | Each creature 

In [41]:
# Test Giant Growth scoring with updated keywords
print("🧪 TESTING GIANT GROWTH WITH UPDATED KEYWORDS")
print("=" * 60)

# Reload the updated constants
import importlib
import src.consts
importlib.reload(src.consts)
from src.consts import theme_keywords, theme_criteria

# Show the updated Stompy keywords
updated_stompy_keywords = theme_keywords['Stompy']
print(f"Updated Stompy keywords: {updated_stompy_keywords}")

# Test Giant Growth scoring
giant_growth_text = "Target creature gets +3/+3 until end of turn."
print(f"\nGiant Growth text: '{giant_growth_text}'")

# Score Giant Growth with updated keywords
theme_score = 0
keyword_matches = []

for keyword in updated_stompy_keywords:
    if keyword.lower() in giant_growth_text.lower():
        theme_score += 1
        keyword_matches.append(keyword)

print(f"\n🎯 GIANT GROWTH STOMPY SCORE:")
print(f"Score: {theme_score}")
print(f"Matching keywords: {keyword_matches}")

# Also test some other pump spells
other_pump_spells = [
    ("Massive Might", "Target creature gets +2/+2 and gains trample until end of turn."),
    ("Rancor", "Enchant creature | Enchanted creature gets +2/+0 and has trample. | When this Aura is put into a graveyard from the battlefield, return it to its owner's hand."),
    ("Colossal Dreadmask", "Living weapon (When this Equipment enters, create a 0/0 black Phyrexian Germ creature token, then attach this to it.) | Equipped creature gets +6/+6 and has trample. | Equip {3}{G}{G}")
]

print(f"\n📊 OTHER PUMP SPELLS SCORING:")
print("-" * 40)

for name, text in other_pump_spells:
    score = 0
    matches = []
    for keyword in updated_stompy_keywords:
        if keyword.lower() in text.lower():
            score += 1
            matches.append(keyword)
    print(f"{name}: Score {score} - {matches}")

print(f"\n✅ IMPROVEMENT SUMMARY:")
print("Before update: Giant Growth scored 0 for Stompy")
print(f"After update: Giant Growth scores {theme_score} for Stompy")
print("This makes pump spells properly recognized as Stompy cards!")

🧪 TESTING GIANT GROWTH WITH UPDATED KEYWORDS
Updated Stompy keywords: ['trample', 'force', 'green', 'attack', 'overrun', 'aggressive', 'gets +', 'target creature', '+1/+1', '+2/+2', '+3/+3', 'until end of turn', 'power', 'combat']

Giant Growth text: 'Target creature gets +3/+3 until end of turn.'

🎯 GIANT GROWTH STOMPY SCORE:
Score: 4
Matching keywords: ['gets +', 'target creature', '+3/+3', 'until end of turn']

📊 OTHER PUMP SPELLS SCORING:
----------------------------------------
Massive Might: Score 5 - ['trample', 'gets +', 'target creature', '+2/+2', 'until end of turn']
Rancor: Score 2 - ['trample', 'gets +']
Colossal Dreadmask: Score 2 - ['trample', 'gets +']

✅ IMPROVEMENT SUMMARY:
Before update: Giant Growth scored 0 for Stompy
After update: Giant Growth scores 4 for Stompy
This makes pump spells properly recognized as Stompy cards!


In [42]:
# Summary and final recommendations
print("📊 FINAL SUMMARY: STOMPY KEYWORD IMPROVEMENTS")
print("=" * 60)

print("🔧 KEYWORDS ADDED TO STOMPY:")
added_keywords = ['gets +', 'target creature', '+1/+1', '+2/+2', '+3/+3', 'until end of turn', 'power', 'combat']
for keyword in added_keywords:
    print(f"  ✅ '{keyword}'")

print(f"\n🎯 IMPACT ON GIANT GROWTH:")
print("  • Before: 0 points (didn't match any Stompy keywords)")  
print("  • After: 4 points (matches 4 keywords)")
print("  • Now properly recognized as a Stompy card!")

print(f"\n🔥 IMPACT ON EXISTING STOMPY CARDS:")
print("  • Massive Might: Now scores 5 points (was probably lower)")
print("  • Rancor: Scores 2 points for pump effect")
print("  • All pump spells now properly score for Stompy theme")

print(f"\n💡 RECOMMENDATION:")
print("=" * 40)
print("1. ✅ KEYWORDS UPDATED - Stompy now recognizes pump spells")
print("2. 🎯 ADD GIANT GROWTH - Will now score 4 points for Stompy")
print("3. 🔄 RUN OPTIMIZATION - Let the algorithm optimize both Green themes")

print(f"\n🚀 NEXT STEPS:")
print("1. Add Giant Growth to Green Stompy deck")
print("2. Run optimize_deck_coherence() to fine-tune the deck")
print("3. Check if any cards should swap between Stompy and Big Creatures")

print(f"\n🏆 EXPECTED RESULT:")
print("Green Stompy should now have better coherence with:")
print("  • Proper scoring for pump spells like Giant Growth")
print("  • Recognition of combat-focused abilities")
print("  • Better distinction from Big Creatures theme")

print(f"\n📈 THEME COHERENCE IMPROVEMENT:")
print("The updated keywords make Stompy more focused on:")
print("  • Pump effects and temporary boosts")
print("  • Combat tricks and aggressive strategies")  
print("  • Power-matters effects")
print("  • Aggressive combat keywords")
print("\nThis should resolve the 'No beneficial swaps found' issue!")

📊 FINAL SUMMARY: STOMPY KEYWORD IMPROVEMENTS
🔧 KEYWORDS ADDED TO STOMPY:
  ✅ 'gets +'
  ✅ 'target creature'
  ✅ '+1/+1'
  ✅ '+2/+2'
  ✅ '+3/+3'
  ✅ 'until end of turn'
  ✅ 'power'
  ✅ 'combat'

🎯 IMPACT ON GIANT GROWTH:
  • Before: 0 points (didn't match any Stompy keywords)
  • After: 4 points (matches 4 keywords)
  • Now properly recognized as a Stompy card!

🔥 IMPACT ON EXISTING STOMPY CARDS:
  • Massive Might: Now scores 5 points (was probably lower)
  • Rancor: Scores 2 points for pump effect
  • All pump spells now properly score for Stompy theme

💡 RECOMMENDATION:
1. ✅ KEYWORDS UPDATED - Stompy now recognizes pump spells
2. 🎯 ADD GIANT GROWTH - Will now score 4 points for Stompy
3. 🔄 RUN OPTIMIZATION - Let the algorithm optimize both Green themes

🚀 NEXT STEPS:
1. Add Giant Growth to Green Stompy deck
2. Run optimize_deck_coherence() to fine-tune the deck
3. Check if any cards should swap between Stompy and Big Creatures

🏆 EXPECTED RESULT:
Green Stompy should now have better co